<h2> CNN Imágenes de Aves </h2>

In [1]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, Input
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.utils import np_utils
import os
import numpy as np
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator 
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

Using TensorFlow backend.


<h2>Configuración de parametros</h2>

In [2]:
width_shape = 224
height_shape = 224
num_classes = 10
batch_size = 32 

<h2>Path de dataset</h2>

In [3]:
train_data_dir = 'dataset/train'  
validation_data_dir = 'dataset/valid'

<h2>Preprocesamiento de imágenes </h2>

In [4]:
train_datagen = ImageDataGenerator(  
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    preprocessing_function=preprocess_input)

valid_datagen = ImageDataGenerator(    
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(  
    train_data_dir,
    target_size=(width_shape, height_shape),
    batch_size=batch_size,
    #save_to_dir='',
    class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(  
    validation_data_dir,
    target_size=(width_shape, height_shape),
    batch_size=batch_size,
    #save_to_dir='',
    class_mode='categorical')


Found 1490 images belonging to 10 classes.
Found 50 images belonging to 10 classes.


<h2>Entrenamiento modelo ResNet50</h2>

In [5]:
from keras.applications.resnet50 import ResNet50

nb_train_samples = 1490
nb_validation_samples = 50


image_input = Input(shape=(width_shape, height_shape, 3))

m_Resnet50 = ResNet50(input_tensor=image_input, include_top=False,weights='imagenet')



last_layer = m_Resnet50.layers[-1].output

x= Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
x=Dropout(0.3)(x)
x = Dense(128, activation='relu', name='fc2')(x)
x=Dropout(0.3)(x)
out = Dense(num_classes, activation='softmax', name='output')(x)
custom_ResNet50_model = Model(image_input, out)


# freeze all the layers except the dense layers
for layer in custom_ResNet50_model.layers[:-6]:
    layer.trainable = False


epochs = 5

custom_ResNet50_model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])


model_history = custom_ResNet50_model.fit_generator(  
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    steps_per_epoch=nb_train_samples//batch_size,
    validation_steps=nb_validation_samples//batch_size)



C:\Users\sanchez\anaconda3\envs\APIDeep\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Epoch 1/5
46/46 [==============================] - 396s 9s/step - loss: 3.8048 - accuracy: 0.4616 - val_loss: 0.6162 - val_accuracy: 0.8438
Epoch 2/5
46/46 [==============================] - 390s 8s/step - loss: 0.8952 - accuracy: 0.7284 - val_loss: 0.0250 - val_accuracy: 1.0000
Epoch 3/5
46/46 [==============================] - 391s 9s/step - loss: 0.6098 - accuracy: 0.8292 - val_loss: 0.3056 - val_accuracy: 0.9375
Epoch 4/5
46/46 [==============================] - 383s 8s/step - loss: 0.4584 - accuracy: 0.8800 - val_loss: 6.0049e-04 - val_accuracy: 1.0000
Epoch 5/5
46/46 [==============================] - 388s 8s/step - loss: 0.3541 - accuracy: 0.9115 - val_loss: 5.4107e-05 - val_accuracy: 1.0000


<h2>Guardar modelo</h2>

In [6]:
custom_ResNet50_model.save("models/model_ResNet50.h5")

<h2>Entrenamiento modelo VGG16</h2>

In [7]:
from keras.applications.vgg16 import VGG16

nb_train_samples = 1490
nb_validation_samples = 50


image_input = Input(shape=(width_shape, height_shape, 3))

m_VGG16 = VGG16(input_tensor=image_input, include_top=True,weights='imagenet')

last_layer = m_VGG16.get_layer('fc2').output
out = Dense(num_classes, activation='softmax', name='output')(last_layer)
custom_VGG16_model = Model(image_input, out)

for layer in custom_VGG16_model.layers[:-1]:
	layer.trainable = False
    

epochs = 5

custom_VGG16_model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])


model_history = custom_VGG16_model.fit_generator(  
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    steps_per_epoch=nb_train_samples//batch_size,
    validation_steps=nb_validation_samples//batch_size)

Epoch 1/5
46/46 [==============================] - 616s 13s/step - loss: 0.3516 - accuracy: 0.9122 - val_loss: 0.0242 - val_accuracy: 1.0000
Epoch 2/5
46/46 [==============================] - 588s 13s/step - loss: 0.0581 - accuracy: 0.9829 - val_loss: 0.0690 - val_accuracy: 0.9444
Epoch 3/5
46/46 [==============================] - 593s 13s/step - loss: 0.0383 - accuracy: 0.9897 - val_loss: 0.0206 - val_accuracy: 1.0000
Epoch 4/5
46/46 [==============================] - 593s 13s/step - loss: 0.0165 - accuracy: 0.9986 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 5/5
46/46 [==============================] - 600s 13s/step - loss: 0.0097 - accuracy: 0.9986 - val_loss: 0.0124 - val_accuracy: 1.0000


<h2>Guardar modelo</h2>

In [8]:
custom_VGG16_model.save("models/model_VGG16.h5")

<h2>Entrenamiento modelo VGG19</h2>

In [9]:
from keras.applications.vgg19 import VGG19


nb_train_samples = 1490
nb_validation_samples = 50


image_input = Input(shape=(width_shape, height_shape, 3))

m_VGG19 = VGG19(input_tensor=image_input, include_top=False,weights='imagenet')



last_layer = m_VGG19.layers[-1].output
x= Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
x=Dropout(0.3)(x)
x = Dense(128, activation='relu', name='fc2')(x)
x=Dropout(0.3)(x)
out = Dense(num_classes, activation='softmax', name='output')(x)
custom_VGG19_model = Model(image_input, out)


# freeze all the layers except the dense layers
for layer in custom_VGG19_model.layers[:-6]:
	layer.trainable = False

epochs = 5
    
custom_VGG19_model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])

model_history = custom_VGG19_model.fit_generator(  
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    steps_per_epoch=nb_train_samples//batch_size,
    validation_steps=nb_validation_samples//batch_size)


Epoch 1/5
46/46 [==============================] - 745s 16s/step - loss: 8.0520 - accuracy: 0.4671 - val_loss: 0.6503 - val_accuracy: 0.8438
Epoch 2/5
46/46 [==============================] - 737s 16s/step - loss: 1.6677 - accuracy: 0.6605 - val_loss: 0.0992 - val_accuracy: 1.0000
Epoch 3/5
46/46 [==============================] - 743s 16s/step - loss: 1.1169 - accuracy: 0.7373 - val_loss: 0.0912 - val_accuracy: 0.9688
Epoch 4/5
46/46 [==============================] - 734s 16s/step - loss: 0.7733 - accuracy: 0.8306 - val_loss: 0.6470 - val_accuracy: 0.8889
Epoch 5/5
46/46 [==============================] - 724s 16s/step - loss: 0.6738 - accuracy: 0.8464 - val_loss: 0.0089 - val_accuracy: 1.0000


<h2>Guardar Modelo</h2>

In [10]:
custom_VGG19_model.save("models/model_VGG19.h5")